In [ ]:
# Version of experiment with common-sense baseline heuristic:
# Defender guards "everywhere" vs attacker taking shortest route to goal

import networkx as nx
import numpy as np
from collections import defaultdict
import itertools

In [ ]:
def find_and_add_entry_node(graph):
    # First identify the original root nodes
    original_roots = [n for n, deg in graph.in_degree() if deg == 0]
    
    if len(original_roots) > 1:
        # add virtual entry node
        entry = 0  # virtual entry node
        graph.add_node(entry)
        for r in original_roots:
            graph.add_edge(entry, r, weight=DEFAULT_WEIGHT_VALUE)
        return entry, graph, original_roots
    else:
        # Only one root, use it as entry
        entry = original_roots[0]
        return entry, graph, original_roots

In [ ]:
def merge_targets_with_multi_edges(orig_graph):
    ## Part 1: This part is concerned only with creating a list of target nodes
    targets = []
    
    # Iterate through all nodes and their out degrees
    for node, out_degree in orig_graph.out_degree():
        # If node has no outgoing edges (degree=0), it's a target
        if out_degree == 0:
            targets.append(node)
    
    ## Part 2: Simple check that just returns the original graph if there are no targets
    if len(targets) <= 1:
        return orig_graph

    # Create just a merged label for the new virtual target node
    merged_label = "c(" + ",".join(str(t) for t in targets) + ")"


    ## Create new MultiDiGraph without edge weights for now
    
    ## Part 3: This part creates a new graph that replaces the original target nodes 
    # with the new virtual target node
    # Immportant: This part does NOT yet add the specific edge weights between the nodes

    # create list that contains all the non-target nodes
    newG = nx.MultiDiGraph()

    # create list that contains all the non-target nodes
    non_targets = []
    for node in orig_graph.nodes():
        if node not in targets:
            non_targets.append(node)
            
    # Add all non-target nodes to new graph
    for node in non_targets:
        newG.add_node(node)
        
    # Add the virtual target node
    newG.add_node(merged_label)
    
    ## Part 4: Edge Recreation between source nodes and the new Virtual Target Node
    # This entire section ensures we maintain all parallel edges and their weights just like R does

    # Track edges 
    pred_target_edges = {}

    ## Part 4a: Collect ALL Edges going to Original Target Nodes

    # We need this info to recreate these edges later with the virtual target node
    # Example of what we're building:
    # If node 5 has these edges:
    # - Edge to node 15 with weight 0.3
    # - Another edge to node 15 with weight 0.7
    # - Edge to node 16 with weight 0.3
    # Then pred_target_edges[5] will contain: [(0.3, '15'), (0.7, '15'), (0.3, '16')]
    for u, v, data in orig_graph.edges(data=True):
        if v in targets:
            if u not in pred_target_edges:
                pred_target_edges[u] = []
            weight = data.get('weight', DEFAULT_WEIGHT_VALUE)
            pred_target_edges[u].append((weight, v))

    ## Part 4b: Count How Many Times Each Weight Appears for Each Source Node

    # For each source node, we count duplicate weights
    # Example: If node 5 has three edges with weights [0.3, 0.7, 0.3]
    # Then weight_counts will be {0.3: 2, 0.7: 1}
    for u, edges in pred_target_edges.items():
        weight_counts = {}
        for weight, _ in edges:
            weight_counts[weight] = weight_counts.get(weight, 0) + 1

        ## Part 4c: For this one source node, finally, create the actual edges to our virtual target
        
        # If weight_counts shows {0.3: 2, 0.7: 1}, we create:
        # - 2 parallel edges with weight 0.3
        # - 1 edge with weight 0.7
        for weight, count in weight_counts.items():
            for _ in range(count):
                newG.add_edge(u, merged_label, weight=weight)
    
    ## Part 5: Copy Over All Other Edges That Don't Touch Target Nodes

    # OK now for the easy part - just copy over all other edges 
    # Example: if we have edge from node 1 -> node 2 with weight 0.5 
    # AND neither node 1 or 2 are target nodes, we just copy it exactly as is

    for u, v, data in orig_graph.edges(data=True):
        # Skip any edges that touch target nodes - we already dealt with those in part 4
        if v not in targets and u not in targets:
            # **data is used to unpacks all attributes automatically
            # So if our edge had data = {'weight': 0.5, 'color': 'red'}
            # This line becomes: newG.add_edge(u, v, weight=0.5, color='red')
            newG.add_edge(u, v, **data)

    return newG

In [ ]:
entry, attack_graph, original_roots = find_and_add_entry_node(attack_graph)
attack_graph = merge_targets_with_multi_edges(attack_graph)

# Find targets manually since your function doesn't return them
target_list = [n for n in attack_graph.nodes() if attack_graph.out_degree(n) == 0]

In [ ]:
# Enumerate all paths (attack strategies) and defense points
def setup_game(attack_graph, entry, target_list):
    target = target_list[0]  # Single target after condensation
    
    # Get all simple paths from entry to target
    routes = list(nx.all_simple_paths(attack_graph, entry, target))
    
    # Get all nodes from all routes
    V = list(set(node for route in routes for node in route))
    
    # Node ordering from topological sort
    try:
        node_order = list(nx.topological_sort(attack_graph))
        node_order = [n for n in node_order if n in V]
    except:
        node_order = V
    
    # Defender action space (exclude trivial nodes)
    if 'as1' not in globals():
        as1 = [n for n in V if n not in [entry, "attacker_entry_node"] + target_list]
    
    as2 = routes  # Attacker action space
    
    # Adversary list (potential starting points)
    adv_list = [n for n in V if n not in [entry] + target_list]
    
    return V, routes, as1, as2, adv_list, target, node_order

V, routes, as1, as2, adv_list, target, node_order = setup_game(attack_graph, entry, target_list)

In [ ]:
# Set default rates if not externally defined
if 'defense_rate_list' not in globals():
    defense_rate_list = [0]
if 'attack_rate_list' not in globals():
    attack_rate_list = [0]

starting_points = [n for n in V if n not in target_list]

In [ ]:
# Main game computation
for defense_rate in defense_rate_list:
    for attack_rate in attack_rate_list:
        
        U = {node: 0.0 for node in V}  # Initialize utility
        
        for i in as1:  # Defense inspection points
            for j in starting_points:  # Attacker starting points
                
                # Get shortest path from j to target
                try:
                    route = nx.shortest_path(attack_graph, j, target)
                except:
                    continue
                
                # Get probability distribution over attacker progress
                pdf_d = random_steps(route, attack_rate, defense_rate, attack_graph)
                
                # Find cut point where defender intercepts
                try:
                    cut_point = route.index(i) + 1  # +1 for 1-based indexing
                except ValueError:
                    cut_point = len(route)
                
                # Calculate payoff distribution
                if sum(pdf_d[:cut_point]) == 0:
                    payoff_distr = [0] * cut_point
                    payoff_distr[cut_point-1] = 1  # Exact cutpoint
                else:
                    # Normalize conditional distribution
                    total = sum(pdf_d[:cut_point])
                    payoff_distr = [p/total for p in pdf_d[:cut_point]]
                
                # Update utility for each node on route
                for idx, node in enumerate(route[:cut_point]):
                    prob = payoff_distr[idx] * (1/len(as1)) * (1/len(V))
                    U[node] += prob
        
        # Output results
        print(f"Defense rate = {defense_rate}, Attack rate = {attack_rate}")
        print(f"Target {target} hit probability: {U[target]:.6f}\n")